<h2 align="center" style="color:blue">Codebasics ML Course: Handling Class Imbalance</h2>

**Dataset credits** UCI Machine Learning Datasets

Downloaded from here: https://archive.ics.uci.edu/dataset/563/iranian+churn+dataset

In [32]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [28]:
df = pd.read_csv("churn.csv")
df.head()

,Call Failure,Complains,Subscription Length,Charge Amount,Seconds of Use,Frequency of use,Frequency of SMS,Distinct Called Numbers,Age Group,Tariff Plan,Status,Age,Customer Value,Churn
0,8,0,38,0,4370,71,5,17,3,1,1,30,197.640,0
1,0,0,39,0,318,5,7,4,2,1,2,25,46.035,0
2,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520,0
3,10,0,38,0,4198,66,1,35,1,1,1,15,240.020,0
4,3,0,38,0,2393,58,2,33,1,1,1,15,145.805,0


In [3]:
df.Churn.value_counts()

Churn
0    2655
1     495
Name: count, dtype: int64

There is a class imbalance. Only 495 customers with Churn value 1 versus 2655 with Churn value 0

In [4]:
df.isna().sum()

Call  Failure              0
Complains                  0
Subscription  Length       0
Charge  Amount             0
Seconds of Use             0
Frequency of use           0
Frequency of SMS           0
Distinct Called Numbers    0
Age Group                  0
Tariff Plan                0
Status                     0
Age                        0
Customer Value             0
Churn                      0
dtype: int64

## Without handling class imbalance

In [50]:
X = df.drop('Churn', axis=1)  
y = df['Churn']  

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=2000)  # Increase max_iter if convergence issues occur
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       531
           1       0.80      0.41      0.55        99

    accuracy                           0.89       630
   macro avg       0.85      0.70      0.74       630
weighted avg       0.88      0.89      0.88       630



## Method 1: Undersampling

In [52]:
y_train.value_counts()

Churn
0    2124
1     396
Name: count, dtype: int64

In [53]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

y_train_rus.value_counts()

Churn
0    396
1    396
Name: count, dtype: int64

In [54]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train_rus, y_train_rus)

y_pred_rus = model.predict(X_test)
report = classification_report(y_test, y_pred_rus)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.81      0.88       531
           1       0.45      0.84      0.58        99

    accuracy                           0.81       630
   macro avg       0.71      0.82      0.73       630
weighted avg       0.88      0.81      0.83       630



## Method 2: SMOTE (Oversampling)

In [56]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
y_train_smote.value_counts()

Churn
0    2124
1    2124
Name: count, dtype: int64

In [57]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train_smote, y_train_smote)

y_pred_smote = model.predict(X_test)
report = classification_report(y_test, y_pred_smote)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.80      0.88       531
           1       0.44      0.85      0.58        99

    accuracy                           0.81       630
   macro avg       0.71      0.83      0.73       630
weighted avg       0.88      0.81      0.83       630



### Method 3: SMOTE Tomek Links (Oversampling)

In [60]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_tomek, y_tomek = smt.fit_resample(X_train, y_train)
y_tomek.value_counts()

Churn
0    2091
1    2091
Name: count, dtype: int64

In [61]:
model = LogisticRegression(max_iter=2000)
model.fit(X_tomek, y_tomek)

y_pred_tomek = model.predict(X_test)
report = classification_report(y_test, y_pred_tomek)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.80      0.88       531
           1       0.45      0.86      0.59        99

    accuracy                           0.81       630
   macro avg       0.71      0.83      0.73       630
weighted avg       0.89      0.81      0.83       630

